In [13]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [14]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '392'
start = '2020-04-01T00:00:00Z'
end = '2021-05-21T00:00:00Z'
mongo_org_collection = db[str(organization_id)]

In [16]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email']},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.type": {"$ne": "null"}
    })
    print(tickets.count())
    file_path = os.path.expanduser('~/Downloads/kyr_field.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket['original_item']
            j = {
                "text_a": i["subject"],
                "text_b": i["description"],
                "created_at": i["created_at"],
                "priority": i['priority'],
                "type": i['type']
            }
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count % 100 == 0:
                print(count)
    print("total: ", count)


get_dataset()

NameError: name 'null' is not defined